In [1]:
!pip install torch torchvision


In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms

In [2]:
import os

In [3]:

# Define the transform for preprocessing the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize with mean and std of ImageNet
                         std=[0.229, 0.224, 0.225])
])

# Load the training dataset
train_dataset = torchvision.datasets.ImageFolder(
    root=r'C:\Users\peter\Github\Chess-voice-recognition\Data_2\Black\train', 
    transform=transform
)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# Load the validation/test dataset
test_dataset = torchvision.datasets.ImageFolder(
    root=r'C:\Users\peter\Github\Chess-voice-recognition\Data_2\Black\val', 
    transform=transform
)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


In [4]:
class VGG16(nn.Module):
    def __init__(self,num_classes=27):
        super(VGG16,self).__init__()
        self.layer1= nn.Sequential(
            nn.Conv2d(3,64, kernel_size = 3, stride =1 , padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(64,64,kernel_size=3, stride =1, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride =2 ))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128, kernel_size = 3, stride= 1 ,padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(128,128,kernel_size = 3, stride =1 , padding =1 ),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(128,256,kernel_size = 3, stride =1 ,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(256,256,kernel_size = 3,stride =1 , padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 =nn.Sequential(
            nn.Conv2d(256,256,kernel_size= 3 ,stride = 1,padding =1 ),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size =2, stride= 2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(256,512,kernel_size =3,stride =1, padding =1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(512,512,kernel_size=3, stride =1 ,padding =1 ),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(512,512,kernel_size = 3,stride =1, padding =1 ),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride =2 ))
        self.layer11 = nn.Sequential(
            nn.Conv2d(512,512,kernel_size= 3, stride =1, padding =1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(512,512,kernel_size=3, stride =1, padding =1 ),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(512,512,kernel_size = 3, stride =1 , padding =1 ),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride = 2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(7*7*512, 4096) ,
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096,4096),
            nn.ReLU())
        self.fc2 = nn.Sequential(
            nn.Linear(4096,num_classes))
    def forward(self,x):
        out= self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out= self.layer4(out)
        out= self.layer5(out)
        out = self.layer6(out)
        out= self.layer7(out)
        out=self.layer8(out)
        out=self.layer9(out)
        out=self.layer10(out)
        out=self.layer11(out)
        out=self.layer12(out)
        out=self.layer13(out)
        out= out.reshape(out.size(0),-1)
        out=self.fc(out)
        out= self.fc1(out)
        out= self.fc2(out)
        return out

In [5]:
model = VGG16(num_classes = 6)

In [6]:
import torch.nn as nn
# from torch.optim import optim
from torch import optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)

In [7]:
def accuracy(out, labels):
  _, pred = torch.max(out, dim=1)
  return torch.sum(pred==labels).item()

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
model = model.to(device)

In [10]:
len(train_loader)

5

In [ ]:
import torch

save_best_model = r'C:/Users/peter/Github/Chess-voice-recognition/weights/weights.pth'
num_epochs = 5
best_acc = 0.0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    train_correct = 0

    for batch_idx, (data, target) in enumerate(train_loader):
        # move the data to the device (GPU or CPU)
        data, target = data.to(device), target.to(device)

        # zero the gradients
        optimizer.zero_grad()

        # forward pass
        output = model(data)

        # calculate the loss
        loss = criterion(output, target)

        # backward pass
        loss.backward()
        optimizer.step()

        # accumulate training accuracy and loss
        train_loss += loss.item() * data.size(0)
        _, predicted = output.max(1)
        train_correct += predicted.eq(target).sum().item()

        # print progress every 10 batches
        if batch_idx % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}] Batch [{batch_idx}/{len(train_loader)}] '
                  f'Train Loss: {train_loss/(batch_idx+1):.6f}')

    # calculate average training accuracy and loss for the epoch
    train_loss /= len(train_loader.dataset)
    train_acc = train_correct / len(train_loader.dataset)
    print(f'Epoch [{epoch+1}/{num_epochs}] Training Loss: {train_loss:.6f}, Training Accuracy: {train_acc:.6f}')

    # evaluate the model on the validation set
    model.eval()
    test_loss = 0
    test_correct = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            # move the data to the device (GPU or CPU)
            data, target = data.to(device), target.to(device)

            # forward pass
            output = model(data)

            # calculate the loss
            loss = criterion(output, target)

            # accumulate test accuracy and loss
            test_loss += loss.item() * data.size(0)
            _, predicted = output.max(1)
            test_correct += predicted.eq(target).sum().item()

    # calculate average test accuracy and loss for the epoch
    test_loss /= len(test_loader.dataset)
    test_acc = test_correct / len(test_loader.dataset)

    # save the best model
    if test_acc > best_acc:
        best_acc = test_acc
        torch.save(model.state_dict(), save_best_model)

    # print the results for the epoch
    print(f'Epoch [{epoch+1}/{num_epochs}] Test Loss: {test_loss:.6f}, Test Accuracy: {test_acc:.6f}')


Epoch [1/5] Batch [0/5] Train Loss: 118.170837
Epoch [1/5] Training Loss: 1.708001, Training Accuracy: 0.343333
Epoch [1/5] Test Loss: 1.791784, Test Accuracy: 0.166667


In [ ]:
xmodel = VGG16(num_classes=27)
optimizer = torch.optim.SGD(vgg.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
save_best_model = '/content/drive/MyDrive/SLC_Project/weights/bestVGG.pth'
for epoch in range(100):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = vgg(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Evaluate the model on the test set
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            outputs = vgg(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()



        test_loss /= len(val_loader.dataset)
        test_acc = test_correct / len(val_loader.dataset)
        if test_acc > best_acc:
          best_acc = test_acc
          torch.save(model.state_dict(), save_best_model)

        print('Epoch [{}/{}], Test Accuracy: {:.2f}%'
              .format(epoch+1, 10, 100 * correct / total))